In [0]:
# model擺放的路徑資料夾
path = ("/content/drive/My Drive/ATSC_model")

In [0]:
# !pip install transformers tqdm boto3 requests regex
import torch
from transformers import BertTokenizer
import pandas as pd
import sys
# 要先將自定義的class(BertForSequenceClassificationReviewsSentiments)的路徑加進sys.path裡，import程式時才找的到檔案
root_path = '/content/drive/My Drive'
folder_path = root_path + '/ATSC_model'
if not root_path in sys.path:
  print('新增路徑')
  sys.path.append(root_path)
if not folder_path in sys.path:
  print('新增路徑')
  sys.path.append(folder_path)
for p in sys.path:
  print(p)
from BertForSequenceClassificationReviewsSentiments import BertForSequenceClassificationReviewsSentiments

In [20]:
%%time
model = torch.load(path + '/model_res_reviews0527.pkl')
tokenizer = BertTokenizer.from_pretrained("bert-base-chinese")

CPU times: user 122 ms, sys: 231 ms, total: 352 ms
Wall time: 719 ms


In [0]:
class ReviewTargetSentimentGenerator:
    global model
    global tokenizer

    def __init__(self, df):
        self.df = df

    def df_tran(self):
    # 先將通過NER model的dataframe轉為ATSC model的input格式 (1.去除沒有菜名的列 2.把有多菜名的列分成多列)
        t = []
        d = []
        empty_dish = (self.df['dish'].isnull())
        df_t = self.df[~empty_dish]
        df_t = df_t.reset_index(drop=True)

        for i in range(len(df_t)):
            dish = df_t.iloc[i,:]["dish"].split(",")
            for j in dish:
                d.append(j) 
            reviews_num = len(dish)
            for k in range(len(dish)):
                t.append(df_t.iloc[i,:]["text"]) 

        dic = {
            "text": t,
            "dish": d
        } 
        df_t = pd.DataFrame(dic, index = None)
        return df_t

    def SentimentGenerator(self):
    # 生成含預測結果的dataframe
        df_all = pd.DataFrame(columns = ["review", "target", "sentiment", "service", "CP", "environment"])        
        for j in range(len(self.df_tran())):
            review = self.df_tran().loc[j, "text"]
            target = self.df_tran().loc[j, "dish"]

            label_tensor = None
            
            word_pieces = ["[CLS]"]
            tokens_review = tokenizer.tokenize(review)
            word_pieces += tokens_review + ["[SEP]"]
            len_review = len(word_pieces)

            tokens_target = tokenizer.tokenize(target)
            word_pieces += tokens_target + ["[SEP]"]
            len_target = len(word_pieces) - len_review

            ids = tokenizer.convert_tokens_to_ids(word_pieces)
            tokens_tensor = torch.tensor(ids)

            segments_tensor = torch.tensor([0] * len_review + [1] * len_target, dtype=torch.long)

            masks_tensor = torch.zeros(tokens_tensor.shape, dtype=torch.long)
            masks_tensor = masks_tensor.masked_fill(tokens_tensor != 0, 1)

            tokens_tensor = tokens_tensor.unsqueeze(0)
            segments_tensor = segments_tensor.unsqueeze(0)
            masks_tensor = masks_tensor.unsqueeze(0)
            data = (tokens_tensor, segments_tensor, masks_tensor)

            device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
            model_r = model.to(device)

            with torch.no_grad():
                # 將所有 tensors 移到 GPU 上
                if next(model_r.parameters()).is_cuda:
                    data = [t.to("cuda:0") for t in data if t is not None]

                tokens_tensors, segments_tensors, masks_tensors = data
                outputs = model_r(input_ids=tokens_tensors, 
                                token_type_ids=segments_tensors, 
                                attention_mask=masks_tensors)    
                logits = outputs[0]
                _, pred = torch.max(logits.data, 1)

            tran = {0: 'negative', 1: 'neutral', 2:'positive'}    
            sentiment = tran[pred.tolist()[0]]
            df_all = df_all.append({"review": review,
                                    "target": target,
                                    "sentiment": sentiment,
                                    "service": 0,
                                    "CP": 0,
                                    "environment": 0}, ignore_index=True)
        return df_all


In [0]:
import pandas as pd
df = pd.read_csv("/content/drive/My Drive/T+阿忠澎湖野生海產_review.csv")
df = df.drop("dish", axis=1)
df.rename(columns={"tag":"dish"}, inplace=True)
df

In [0]:
df = ReviewTargetSentimentGenerator(df)
df_result = df.SentimentGenerator()
df_result